<a href="https://colab.research.google.com/github/Annie00000/Project/blob/main/12_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. 繪製各svid圖 (找範例)

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import os

# 假設您的 DataFrame 命名為 df
# 確保您的 df 已經載入

# 1. 找出需要繪圖的欄位
cols_to_exclude = ['col1', 'col2', 'col3', 'time']
cols_to_plot = [col for col in df.columns if col not in cols_to_exclude]


# 2. 遍歷欄位並繪製/儲存圖表
output_files = []
for col in cols_to_plot:
    # 建立 go.Figure() 並加入 go.Scatter() trace
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=df['time'],
        y=df[col],
        mode='markers', # 使用 'markers' 顯示點狀圖
        name=col
    ))

    # 設定標題和軸標籤
    fig.update_layout(
        title=f'{col} 的時間序列圖',
        xaxis_title='time',
        yaxis_title=col
    )

    # 儲存圖表為 HTML 檔案
    filename = f'plot_{col}.html'
    fig.write_html(filename)
    output_files.append(filename)

print(f"圖表已儲存為 HTML 檔案: {output_files}")

## 2. rule 4 : 連續 n 點上升

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from typing import List, Tuple, Dict, Optional

# --- 1. 數據準備 (模擬數據 - 與前次相同) ---
data = {
    'ctrljob': ['A', 'A', 'A', 'A', 'B', 'B', 'B', 'C', 'C', 'C', 'D', 'D', 'D', 'E', 'E', 'F', 'F'],
    'claim_time': range(17),
    'svid': [10.2, 11.5, 12.0, 15.5,  # A: 連續上升 4 點
             9.0, 8.5, 1.0,           # B: 極值
             10.0, 10.3, 10.5,        # C: 連續上升 3 點
             10.0, 10.1, 10.0,        # D: 穩定
             9.9, 10.2,               # E: 穩定
             10.0, 10.1],             # F: 穩定
    'group': ['Bad'] * 10 + ['Good'] * 7
}
df = pd.DataFrame(data)

# -----------------------------------------------------
# ✨ 核心函式 (Rule 5 特徵提取 - 必須先執行以找到最佳 n)
# -----------------------------------------------------

def get_longest_rising_sequence_info(series: pd.Series) -> Tuple[int, Optional[int]]:
    """計算序列中最長連續上升序列長度 (n) 和該序列在原始 DataFrame 中的起始索引。"""
    if series.empty or len(series) < 2:
        return 0, None

    is_rising = series.diff().iloc[1:] > 0
    original_indices = series.index.tolist()

    max_len = 0
    max_start_index = None
    current_count = 0

    for i, is_one in enumerate(is_rising):
        if is_one:
            current_count += 1
        else:
            if current_count + 1 > max_len and current_count >= 1:
                max_len = current_count + 1
                max_start_index = original_indices[i + 1 - current_count]

            current_count = 0

    if current_count + 1 > max_len and current_count >= 1:
        max_len = current_count + 1
        max_start_index = original_indices[len(series) - max_len]

    return (max_len, max_start_index) if max_len >= 2 else (0, None)

def get_accuracy(true_groups: pd.Series, predicted_groups: pd.Series) -> float:
    """計算準確度"""
    return accuracy_score(true_groups, predicted_groups)

# -----------------------------------------------------
# 2. 確定最佳折衷 n_critical
# -----------------------------------------------------

# 特徵提取: 找出每個 ctrljob 最長的連續上升長度 (n)
r5_features = df.groupby('ctrljob').agg(
    Rising_Info=('svid', get_longest_rising_sequence_info),
    True_Group=('group', 'first')
).reset_index()

r5_features[['Max_Rising_Len', 'Start_Index']] = pd.DataFrame(
    r5_features['Rising_Info'].tolist(),
    index=r5_features.index
)

# 定義 n_critical 的搜索範圍
max_len_in_data = int(r5_features['Max_Rising_Len'].max())
n_critical_range: List[int] = list(range(2, max_len_in_data + 1))

r5_results: List[Dict] = []
for n_critical in n_critical_range:
    r5_features['Predicted_Group'] = np.where(
        r5_features['Max_Rising_Len'] >= n_critical,
        'Bad',
        'Good'
    )

    current_accuracy = get_accuracy(r5_features['True_Group'], r5_features['Predicted_Group'])
    r5_results.append({'n_critical': n_critical, 'Accuracy': current_accuracy})

r5_df = pd.DataFrame(r5_results).sort_values(by='Accuracy', ascending=False)

# 🏆 折衷策略邏輯：選擇最高準確度範圍內的中間 n
max_accuracy_r5 = r5_df['Accuracy'].max()
best_r5_candidates = r5_df[r5_df['Accuracy'] == max_accuracy_r5]

n_min = best_r5_candidates['n_critical'].min()
n_max = best_r5_candidates['n_critical'].max()
n_compromise = int(np.round((n_min + n_max) / 2))

print(f"Rule 5 最佳準確度: {max_accuracy_r5:.4f}")
print(f"達到該準確度的 n 範圍: {n_min} 到 {n_max}")
print(f"依折衷策略選擇的最佳 n: **{n_compromise}**")
print("-" * 50)

# -----------------------------------------------------
# 3. 點對點判定函式 (使用 n_compromise)
# -----------------------------------------------------

def mark_rule_5_points_with_n(df: pd.DataFrame, svid_col: str, n_critical: int) -> pd.Series:
    """
    使用最佳 n_critical，標記 df 中每個點是否屬於長度 >= n_critical 的連續上升序列。
    """
    if n_critical < 2:
        return pd.Series('PASS', index=df.index) # 確保回傳值為 'PASS'

    r5_status = pd.Series('PASS', index=df.index)

    # 必須根據 ctrljob 分組檢查
    for job_id, job_data in df.groupby('ctrljob'):
        series = job_data[svid_col]

        if len(series) < n_critical:
            continue

        is_rising = series.diff().iloc[1:] > 0
        original_indices = series.index.tolist()

        current_count = 0 # 連續上升次數 (n-1)

        # 遍歷上升判斷序列
        for i, is_one in enumerate(is_rising):
            if is_one:
                current_count += 1
            else:
                # 序列中斷，檢查前一個序列
                sequence_len = current_count + 1
                if sequence_len >= n_critical:
                    # 標記整個符合條件的序列 (長度 sequence_len)
                    start_loc = i + 1 - sequence_len
                    end_loc = i + 1 # exclusive end

                    r5_status.loc[original_indices[start_loc : end_loc]] = 'FAIL'

                current_count = 0

        # 檢查序列末尾
        sequence_len = current_count + 1
        if sequence_len >= n_critical:
            start_loc = len(series) - sequence_len
            end_loc = len(series)

            r5_status.loc[original_indices[start_loc : end_loc]] = 'FAIL'

    return r5_status

# -----------------------------------------------------
# 4. 應用並輸出結果
# -----------------------------------------------------

# 應用最佳 n
df['Rule5_Status'] = mark_rule_5_points_with_n(df, 'svid', n_compromise)

# 輸出最終點位判定結果
final_result = df[['ctrljob', 'claim_time', 'svid', 'group', 'Rule5_Status']].copy()

print(f"## 每個點位 Rule 5 判定結果 (使用 n={n_compromise})")
print(final_result.to_markdown(index=False, floatfmt=".2f"))

### rule 4  改用 Decision Tree 來決定 n (用有使用過的n)

**替代參數搜索 (遍歷) 的方法**


> 基於分佈分析的臨界值 (統計方法):


  將兩邊連續 $x$ 點上升的 $x$ 數，各自做成一個分佈，然後取 $Good$ 的 $x$ 最大
值與 $Bad$ 分佈的交界」，這指向了尋找最佳分類邊界

* 步驟：

  1. 特徵提取 ($x$): 針對每個 ctrljob，計算其「最長連續上升點數 ($x$)」。
  2. 分離分佈： 獲得 $Good$ 群組的 $x$ 分佈 ($X_{Good}$) 和 $Bad$ 群組的 $x$ 分佈 ($X_{Bad}$)。
  3. 尋找最佳分割點 ($n$): 找到一個臨界值 $n$，使得分類結果：
    - $\text{IF } x \ge n \text{ THEN Bad}$
    - $\text{IF } x < n \text{ THEN Good}$
    * 能夠最大化分類準確度。這本質上就是單一特徵的最佳閾值搜索。



> 如何找到 $n$？

* **優化替代**： 您可以使用 ROC 曲線或 F1-Score 作為優化指標，來代替單純的 $Accuracy$。雖然這仍然是遍歷 $n$ 的可能取值範圍，但搜索範圍被限制在實際數據中出現過的 $x$ 值，而不是任意的 $n$ 範圍，效率更高。

* **決策樹 (Decision Tree)**： 最簡單且最有效的方法之一。(單規則)

  1. 建立一個僅使用 Max_Rising_Len ($x$) 作為輸入特徵的決策樹模型，目標是預測 Group (Bad/Good)。
  2. 由於只有一個特徵，決策樹將會學習到一個或多個最佳的單邊臨界值來分割數據。
  3. 輸出： 決策樹的第一個分割點就是根據 $Gini$ 或 $Entropy$ 標準找到的，能帶來最大資訊增益的最佳臨界值 $n$。

  -  Gini Impurity

In [ ]:
from sklearn.tree import DecisionTreeClassifier, export_text

# ... (Rule 5 特徵提取部分同前，生成 r5_features) ...

# 準備數據 for 決策樹
X = r5_features[['Max_Rising_Len']] # 輸入特徵 (x)
y = r5_features['True_Group']       # 目標 (Good/Bad)

# 初始化決策樹模型 (深度設為 1，強制只找一個分割點)
tree_model = DecisionTreeClassifier(max_depth=1, random_state=42)
tree_model.fit(X, y)

# 提取最佳分割點
# 決策樹的節點屬性中包含分割點
best_n_split = tree_model.tree_.threshold[0]

# 由於決策樹可能在兩點之間分割，我們取整數作為臨界值
n_optimal_tree = int(np.ceil(best_n_split))

# 輸出結果
print(f"決策樹找到的最佳臨界值 n: {n_optimal_tree}")

**最佳參數選擇方法**

1. $n_{\text{compromise}}$ :
  - 遍歷法 (計算 $N_{\text{optimal}}$ 的中點)
  - 最平衡。它代表了在不犧牲準確度的前提下，敏感度與穩健性的折衷點。

2. $n_{\text{tree}}$ :
  - 決策樹法 (Gini Impurity 最優)
  - 最數學/統計最佳。它是根據資訊理論或 Gini 標準，將 $Good/Bad$ 數據分得最「純淨」的那個臨界值。

3. $n_{\text{min}}$ :
  - 遍歷法 (計算 $N_{\text{optimal}}$ 的最小值)
  - 最敏感的 SPC 規則。它能提供最快的異常警報。

## 3. Rule 5 : 連續n點下降

In [ ]:
## 原版


# -----------------------------------------------------
# ✨ 核心函式 (Rule 5 特徵提取 - 必須先執行以找到最佳 n) 最長連續下降長度
# -----------------------------------------------------

from typing import Optional, Tuple

def get_longest_falling_sequence_info(series: pd.Series) -> Tuple[int, Optional[int]]:
    """
    Rule 6 特徵提取: 計算序列中出現過的最長的連續下降序列長度 (n)
    和該序列在原始 DataFrame 中的起始索引。
    """
    if series.empty or len(series) < 2:
        return 0, None

    # 關鍵：檢查是否為單點下降 (當前點 < 前一點)
    is_falling = series.diff().iloc[1:] < 0
    original_indices = series.index.tolist()

    max_len = 0
    max_start_index = None
    current_count = 0    # 記錄連續下降的次數 (n-1)

    for i, is_one in enumerate(is_falling):
        if is_one:
            current_count += 1
        else:
            if current_count + 1 > max_len and current_count >= 1:
                max_len = current_count + 1
                max_start_index = original_indices[i + 1 - current_count]

            current_count = 0

    # 處理序列末尾的連續下降
    if current_count + 1 > max_len and current_count >= 1:
        max_len = current_count + 1
        max_start_index = original_indices[len(series) - max_len]

    return (max_len, max_start_index) if max_len >= 2 else (0, None)




In [ ]:
# -----------------------------------------------------
# 2. 確定最佳折衷 n_critical
# -----------------------------------------------------

# --- C. Rule 6 獨立優化 (搜索最佳 n_critical) ---

# 1. 特徵提取: 找出每個 ctrljob 最長連續下降長度 (x)
r6_features = df.groupby('ctrljob').agg(
    Falling_Info=('svid', get_longest_falling_sequence_info),
    True_Group=('group', 'first')
).reset_index()

r6_features[['Max_Falling_Len', 'Start_Index']] = pd.DataFrame(
    r6_features['Falling_Info'].tolist(),
    index=r6_features.index
)

# 2. 定義 n_critical 的搜索範圍
max_len_in_data_r6 = int(r6_features['Max_Falling_Len'].max())
n_critical_range_r6: List[int] = list(range(2, max_len_in_data_r6 + 1))

r6_results: List[Dict] = []
for n_critical in n_critical_range_r6:
    # 判定：如果 Max_Falling_Len >= n_critical，則判定為 Bad
    r6_features['Predicted_Group'] = np.where(
        r6_features['Max_Falling_Len'] >= n_critical,
        'Bad',
        'Good'
    )

    current_accuracy = get_accuracy(r6_features['True_Group'], r6_features['Predicted_Group'])
    r6_results.append({'n_critical': n_critical, 'Accuracy': current_accuracy})

# 3. 🏆 應用折衷策略
r6_df = pd.DataFrame(r6_results).sort_values(by='Accuracy', ascending=False)
max_accuracy_r6 = r6_df['Accuracy'].max()
best_r6_candidates = r6_df[r6_df['Accuracy'] == max_accuracy_r6]

if len(best_r6_candidates) > 0:
    n_min_r6 = best_r6_candidates['n_critical'].min()
    n_max_r6 = best_r6_candidates['n_critical'].max()
    n_compromise_r6 = int(np.round((n_min_r6 + n_max_r6) / 2))
else:
    # 處理沒有結果的特殊情況
    n_compromise_r6 = 0

In [ ]:
# -----------------------------------------------------
# 3. 點對點判定函式 (使用 n_compromise)
# -----------------------------------------------------
def mark_rule_6_points_with_n(df: pd.DataFrame, svid_col: str, n_critical: int) -> pd.Series:
    """
    使用最佳 n_critical，標記 df 中每個點是否屬於長度 >= n_critical 的連續下降序列。
    """
    if n_critical < 2:
        return pd.Series('PASS', index=df.index)

    r6_status = pd.Series('PASS', index=df.index)

    for job_id, job_data in df.groupby('ctrljob'):
        series = job_data[svid_col]

        if len(series) < n_critical:
            continue

        # 關鍵：檢查下降趨勢
        is_falling = series.diff().iloc[1:] < 0
        original_indices = series.index.tolist()

        current_count = 0

        for i, is_one in enumerate(is_falling):
            if is_one:
                current_count += 1
            else:
                # 序列中斷，檢查前一個序列
                sequence_len = current_count + 1
                if sequence_len >= n_critical:
                    # 標記整個符合條件的序列
                    start_loc = i + 1 - sequence_len
                    end_loc = i + 1

                    r6_status.loc[original_indices[start_loc : end_loc]] = 'FAIL'

                current_count = 0

        # 檢查序列末尾
        sequence_len = current_count + 1
        if sequence_len >= n_critical:
            start_loc = len(series) - sequence_len
            end_loc = len(series)

            r6_status.loc[original_indices[start_loc : end_loc]] = 'FAIL'

    return r6_status

## 4. 整合 Rule 4/5 的通用程式碼

In [ ]:
# -----------------------------------------------------
# ✨ 通用特徵提取
# -----------------------------------------------------

# 計算最長連續序列的長度和位置，它接受一個 check_func 參數來定義趨勢方向
from typing import Callable, Tuple, Optional
# 趨勢檢查函式的類型：輸入 Series (svid), 輸出 Series (True/False)
TrendCheckFunc = Callable[[pd.Series], pd.Series]

def get_consecutive_sequence_info(series: pd.Series, check_func: TrendCheckFunc) -> Tuple[int, Optional[int]]:
    """
    通用特徵提取: 計算序列中出現過的最長的連續序列長度 (n) 和該序列的起始索引。

    :param series: 單一 ctrljob 的 svid 序列。
    :param check_func: 判斷趨勢的函式 (e.g., lambda s: s.diff().iloc[1:] > 0)
    :return: (最長連續點數, 起始索引)
    """
    if series.empty or len(series) < 2:
        return 0, None

    # 關鍵：使用傳入的函式判斷趨勢 (上升或下降)
    is_trend = check_func(series)
    original_indices = series.index.tolist()

    max_len = 0
    max_start_index = None
    current_count = 0    # 連續趨勢次數 (n-1)

    # 遍歷趨勢判斷序列
    for i, is_one in enumerate(is_trend):
        if is_one:
            current_count += 1
        else:
            if current_count + 1 > max_len and current_count >= 1:
                max_len = current_count + 1
                max_start_index = original_indices[i + 1 - current_count]

            current_count = 0

    # 處理序列末尾
    if current_count + 1 > max_len and current_count >= 1:
        max_len = current_count + 1
        max_start_index = original_indices[len(series) - max_len]

    return (max_len, max_start_index) if max_len >= 2 else (0, None)

In [ ]:
# -----------------------------------------------------
# 通用點位標記函式 (mark_consecutive_points)
# -----------------------------------------------------

# 最佳 $n$ 應用到點位，標記每個點是 FAIL 還是 PASS

def mark_consecutive_points(df: pd.DataFrame, svid_col: str, n_critical: int, check_func: TrendCheckFunc) -> pd.Series:
    """
    通用點位標記: 標記 df 中每個點是否屬於長度 >= n_critical 的連續序列。
    """
    if n_critical < 2:
        return pd.Series('PASS', index=df.index)

    r_status = pd.Series('PASS', index=df.index)

    for job_id, job_data in df.groupby('ctrljob'):
        series = job_data[svid_col]

        if len(series) < n_critical:
            continue

        # 關鍵：使用傳入的函式判斷趨勢
        is_trend = check_func(series)
        original_indices = series.index.tolist()

        current_count = 0

        for i, is_one in enumerate(is_trend):
            if is_one:
                current_count += 1
            else:
                sequence_len = current_count + 1
                if sequence_len >= n_critical:
                    # 標記整個符合條件的序列
                    start_loc = i + 1 - sequence_len
                    end_loc = i + 1
                    r_status.loc[original_indices[start_loc : end_loc]] = 'FAIL'

                current_count = 0

        # 檢查序列末尾
        sequence_len = current_count + 1
        if sequence_len >= n_critical:
            start_loc = len(series) - sequence_len
            end_loc = len(series)
            r_status.loc[original_indices[start_loc : end_loc]] = 'FAIL'

    return r_status

In [ ]:
# --- 定義趨勢檢查函式 ---
rising_check_func = lambda s: s.diff().iloc[1:] > 0
falling_check_func = lambda s: s.diff().iloc[1:] < 0


# --- Rule 5 (上升) 獨立優化 ---
r5_features = df.groupby('ctrljob').agg(
    Rising_Info=('svid', lambda s: get_consecutive_sequence_info(s, rising_check_func)),
    True_Group=('group', 'first')
).reset_index()
# ... 接著執行折衷 n 的搜索，得到 n_compromise_r5 ...

# --- Rule 6 (下降) 獨立優化 ---
r6_features = df.groupby('ctrljob').agg(
    Falling_Info=('svid', lambda s: get_consecutive_sequence_info(s, falling_check_func)),
    True_Group=('group', 'first')
).reset_index()
# ... 接著執行折衷 n 的搜索，得到 n_compromise_r6 ...


# --- 應用到點位判定 ---
# df['Rule5_Status'] = mark_consecutive_points(df, 'svid', n_compromise_r5, rising_check_func)
# df['Rule6_Status'] = mark_consecutive_points(df, 'svid', n_compromise_r6, falling_check_func)

## 5. 呈上，通用 rule 4/5 + Decision Tree

In [ ]:

# -----------------------------------------------------
# 核心 DT 優化函式
# -----------------------------------------------------
# 使用深度為 1 的決策樹來找到最佳分割點 $n$。

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

def find_optimal_n_via_dt(features_df: pd.DataFrame, feature_col: str) -> Tuple[int, float]:
    """
    使用決策樹 (Decision Tree, max_depth=1) 自動找到單一特徵的最佳分割臨界值 n。

    :param features_df: 包含 Max_Consecutive_Len 和 True_Group 的 DataFrame。
    :param feature_col: 特徵欄位名稱 (e.g., 'Max_Rising_Len')。
    :return: (最佳臨界值 n, 該 n 達成的 Accuracy)
    """
    # 準備數據
    X = features_df[[feature_col]].fillna(0) # 填補 NaN 以防決策樹報錯
    y = features_df['True_Group']

    # 1. 訓練決策樹 (max_depth=1 強制只找一個分割點)
    tree_model = DecisionTreeClassifier(max_depth=1, random_state=42)
    tree_model.fit(X, y)

    # 2. 提取最佳分割點 (threshold)
    # best_n_split_raw 是決策樹找到的最佳分割值 (可能帶小數)
    best_n_split_raw = tree_model.tree_.threshold[0]

    # 3. 確定整數臨界值 n
    # 若分割點為 3.5，則 n >= 4 為 Bad；若分割點為 3.0，則 n > 3 亦即 n >= 4 為 Bad
    # 使用 ceil 確保臨界值 n 符合 'x >= n THEN Bad' 的邏輯
    n_optimal = int(np.ceil(best_n_split_raw))

    # 4. 應用該 n 並計算 Accuracy (作為模型的性能指標)
    features_df['Predicted_Group'] = np.where(
        features_df[feature_col] >= n_optimal,
        'Bad',
        'Good'
    )
    accuracy = accuracy_score(y, features_df['Predicted_Group'])

    return n_optimal, accuracy




In [ ]:
# -----------------------------------------------------
# Rule 5 和 Rule 6 的應用範例
# -----------------------------------------------------


# --- 定義趨勢檢查函式 (Rule 5/6 共用) ---
rising_check_func = lambda s: s.diff().iloc[1:] > 0
falling_check_func = lambda s: s.diff().iloc[1:] < 0


# -----------------------------------------------
# A. Rule 5 優化 (連續上升)
# -----------------------------------------------

# 1. 提取特徵
r5_features = df.groupby('ctrljob').agg(
    Rising_Info=('svid', lambda s: get_consecutive_sequence_info(s, rising_check_func)),
    True_Group=('group', 'first')
).reset_index()
r5_features[['Max_Rising_Len', 'Start_Index']] = pd.DataFrame(
    r5_features['Rising_Info'].tolist(),
    index=r5_features.index
)

# 2. 應用決策樹尋找最佳 n
n_optimal_r5, accuracy_r5 = find_optimal_n_via_dt(r5_features.copy(), 'Max_Rising_Len')

print("--- Rule 5 決策樹優化結果 ---")
print(f"最佳臨界值 n (決策樹): {n_optimal_r5}")
print(f"該 n 達成的準確度: {accuracy_r5:.4f}")


# -----------------------------------------------
# B. Rule 6 優化 (連續下降)
# -----------------------------------------------

# 1. 提取特徵
r6_features = df.groupby('ctrljob').agg(
    Falling_Info=('svid', lambda s: get_consecutive_sequence_info(s, falling_check_func)),
    True_Group=('group', 'first')
).reset_index()
r6_features[['Max_Falling_Len', 'Start_Index']] = pd.DataFrame(
    r6_features['Falling_Info'].tolist(),
    index=r6_features.index
)

# 2. 應用決策樹尋找最佳 n
n_optimal_r6, accuracy_r6 = find_optimal_n_via_dt(r6_features.copy(), 'Max_Falling_Len')

print("\n--- Rule 6 決策樹優化結果 ---")
print(f"最佳臨界值 n (決策樹): {n_optimal_r6}")
print(f"該 n 達成的準確度: {accuracy_r6:.4f}")


# -----------------------------------------------
# 3. 應用到點位判定
# -----------------------------------------------
# df['Rule5_Status'] = mark_consecutive_points(df, 'svid', n_optimal_r5, rising_check_func)
# df['Rule6_Status'] = mark_consecutive_points(df, 'svid', n_optimal_r6, falling_check_func)

**總結與注意事項**：

- 效率提升: 使用決策樹後，您不再需要手動遍歷 $n_{\text{critical}}$ 範圍，程式碼更加精簡高效。
- 單一解： 正如我們討論過的，決策樹只會返回一個 Gini 最優的分割點 $n_{\text{optimal}}$。如果您需要**「折衷」或「最敏感」**的 $n$，您必須回到手動遍歷 $Accuracy=1.0$ 的範圍進行選擇。
- DT 內建指標： 決策樹預設使用 Gini Impurity 作為其優化指標，以確保找到的分割點能使分類子集達到最高的純淨度。